# 📊 Análise de Segmentação de Clientes

## 🧩 Contexto dos Dados e Problema a Ser Resolvido

Este estudo foi conduzido com dados de clientes de uma empresa, com o objetivo principal de identificar diferentes perfis de comportamento de compra. A partir dessa segmentação, busca-se personalizar campanhas de marketing de forma mais eficiente.

Os dados utilizados contemplam três dimensões principais do comportamento do consumidor:

- **Recência**: Quanto tempo se passou desde a última compra.
- **Frequência**: Quantidade de vezes que o cliente realizou compras.
- **Valor Monetário**: Total gasto pelo cliente em compras.

### 🎯 Problema de Negócio

**Como otimizar campanhas de marketing para melhorar a retenção de clientes, aumentar o engajamento e elevar o LTV (Lifetime Value)?**

A resposta para essa pergunta passa pela segmentação inteligente da base de clientes, permitindo estratégias de marketing mais direcionadas, eficientes e com maior retorno sobre o investimento.

## 🛠️ Técnicas Utilizadas

Neste projeto, utilizamos a **análise RFM** (*Recência, Frequência e Valor Monetário*) como técnica principal para segmentar os clientes com base em seu comportamento de compra.

A metodologia RFM permite agrupar os clientes de acordo com:

- **Recência**: Quanto tempo se passou desde a última compra.
- **Frequência**: Quantidade total de compras realizadas.
- **Valor Monetário**: Total gasto pelo cliente durante o período analisado.

Essa abordagem fornece uma visão estratégica dos diferentes perfis de consumidores, permitindo a criação de **campanhas de marketing personalizadas** e mais eficazes para cada segmento.

## Importando bibliotecas necessárias

In [2]:
import pandas as pd
import numpy as np

## Análise Exploratória (EDA)

### Visualização inicial dos dados

In [3]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
df = pd.read_excel(url)

In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Verificação dos atributos e tamanho do dataset

In [6]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [7]:
df.shape

(541909, 8)

### Tipos de dados e valores ausentes

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [10]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

## Pré-Processamento

### Tratamento de dados faltantes

In [11]:
df['Description'] = df['Description'].fillna('Sem descrição')
df.dropna(subset=['CustomerID'], inplace=True)
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

### Conversão da coluna de datas e cálculo do TotalSpend

In [16]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalSpend'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSpend
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


## Métricas RFM

### Cálculo da Recência (R)

In [17]:
latest_date = df['InvoiceDate'].max()
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days
})
rfm.rename(columns={'InvoiceDate': 'Recency'}, inplace=True)
rfm.head()

,Recency
CustomerID,
12346.0,325
12347.0,1
12348.0,74
12349.0,18
12350.0,309


### Cálculo da Frequência (F)

In [18]:
rfm['Monetary'] = df.groupby('CustomerID')['TotalSpend'].sum()
rfm.head()

,Recency,Monetary
CustomerID,,
12346.0,325,0.00
12347.0,1,4310.00
12348.0,74,1797.24
12349.0,18,1757.55
12350.0,309,334.40


### Cálculo do Valor Monetário (M)

In [19]:
rfm['Monetary'] = df.groupby('CustomerID')['TotalSpend'].sum()
rfm.head()

,Recency,Monetary
CustomerID,,
12346.0,325,0.00
12347.0,1,4310.00
12348.0,74,1797.24
12349.0,18,1757.55
12350.0,309,334.40


## Cálculo Final e Modelagem RFM

### Cálculo completo do RFM com nova referência de data

In [20]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalPrice': 'sum'
})

rfm.rename(columns={'InvoiceDate': 'Recencia', 'InvoiceNo': 'Frequencia', 'TotalPrice': 'Valor'}, inplace=True)
rfm.head()

,Recencia,Frequencia,Valor
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


## Score e Segmentação

### Atribuição de scores R, F e M

In [21]:
rfm['Recencia_Score'] = pd.qcut(rfm['Recencia'], 5, labels=[5, 4, 3, 2, 1])
rfm['Frequencia_Score'] = pd.qcut(rfm['Frequencia'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
rfm['Valor_Score'] = pd.qcut(rfm['Valor'], 5, labels=[1, 2, 3, 4, 5], duplicates='drop')

rfm['Total_Score'] = rfm['Recencia_Score'].astype(int) + rfm['Frequencia_Score'].astype(int) + rfm['Valor_Score'].astype(int)
rfm.head()

,Recencia,Frequencia,Valor,Recencia_Score,Frequencia_Score,Valor_Score,Total_Score
CustomerID,,,,,,,
12346.0,326,2,0.00,1,1,1,3
12347.0,2,182,4310.00,5,5,5,15
12348.0,75,31,1797.24,2,3,4,9
12349.0,19,73,1757.55,4,4,4,12
12350.0,310,17,334.40,1,2,2,5


### Criação das categorias de segmentação

In [22]:
def segmentacao(row):
    if row['Total_Score'] >= 13:
        return 'Segmento 1: Clientes mais valiosos'
    elif row['Total_Score'] >= 9:
        return 'Segmento 2: Clientes potenciais'
    elif row['Total_Score'] >= 5:
        return 'Segmento 3: Clientes regulares'
    else:
        return 'Segmento 4: Clientes inativos'

rfm['Segmento'] = rfm.apply(segmentacao, axis=1)
rfm.head()

,Recencia,Frequencia,Valor,Recencia_Score,Frequencia_Score,Valor_Score,Total_Score,Segmento
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,1,1,3,Segmento 4: Clientes inativos
12347.0,2,182,4310.00,5,5,5,15,Segmento 1: Clientes mais valiosos
12348.0,75,31,1797.24,2,3,4,9,Segmento 2: Clientes potenciais
12349.0,19,73,1757.55,4,4,4,12,Segmento 2: Clientes potenciais
12350.0,310,17,334.40,1,2,2,5,Segmento 3: Clientes regulares


### Salvando os Resultados

In [23]:
rfm.to_csv('../data/processed/rfm_segmentation.csv')

## Resultados

### 🎯 Estratégias de Marketing com base na Segmentação RFM

Com base na tabela RFM (*Recência*, *Frequência* e *Valor Monetário*), é possível aplicar várias campanhas de marketing segmentadas para melhorar o engajamento dos clientes. Abaixo estão algumas sugestões de campanhas para cada segmento identificado:

---

#### 🥇 Segmento 1: Clientes mais valiosos
**Campanhas:**
- **Fidelidade:** Recompensas exclusivas para manter o engajamento.
- **Ofertas personalizadas:** Descontos em produtos premium.
- **Agradecimento:** Cartões de agradecimento ou bônus de compras.
- **Acesso antecipado:** Acesso privilegiado a novos produtos ou promoções.

---

#### 🚀 Segmento 2: Clientes potenciais
**Campanhas:**
- **Reengajamento:** Cupons ou promoções para incentivar novas compras.
- **Promoções de frequência:** Descontos progressivos para compras recorrentes.
- **Novidades:** E-mails com lançamentos e novos produtos.
- **Recomendação:** Benefícios para indicações de amigos.

---

#### 🔁 Segmento 3: Clientes regulares
**Campanhas:**
- **Incentivo a mais compras:** Promoções específicas para aumentar a frequência.
- **Valor agregado:** Descontos para compras acima de um determinado valor.
- **Cross-selling:** Ofertas de produtos complementares com base no histórico.

---

#### 💤 Segmento 4: Clientes inativos
**Campanhas:**
- **Reativação:** Descontos ou ofertas exclusivas para trazer o cliente de volta.
- **Recuperação de carrinho:** Lembretes para finalizar compras abandonadas.
- **Novos produtos:** E-mails com sugestões baseadas em compras anteriores.

---

#### 🧊 Segmento 5: Clientes de baixo valor
**Campanhas:**
- **Incentivo à primeira compra:** Descontos para aumentar o ticket médio.
- **Despertar interesse:** Ofertas introdutórias ou bônus


### ✅ Conclusão

A análise de segmentação de clientes por meio da metodologia RFM (Recência, Frequência e Valor Monetário) permitiu identificar padrões distintos de comportamento entre os clientes. A partir dessa segmentação, foi possível propor campanhas de marketing personalizadas, com o objetivo de aumentar o engajamento e a fidelização.

As estratégias sugeridas abrangem desde programas de fidelidade e ofertas exclusivas para os clientes mais valiosos, até ações de reativação voltadas para aqueles que estão inativos. Essa abordagem segmentada permite otimizar os recursos de marketing, tornando as campanhas mais eficazes, relevantes e direcionadas às necessidades específicas de cada grupo de clientes.

Com a aplicação prática dessas campanhas, espera-se não apenas um aumento nas vendas, mas também o fortalecimento do relacionamento com os clientes e a construção de uma base mais sólida e lucrativa ao longo do tempo.
